In [1]:
import helper

In [2]:
enzyme_datafile = 'Data/Modified-Enzyme-Alignment.txt'

X,y,enz_names = helper.parseEnzymeFile(enzyme_datafile)

In [3]:
from FeatEngg.ctdEnc import CTD

In [4]:
from mySVM.model import SVM

In [33]:
ctdEnc = CTD(F='C')

In [34]:
import numpy as np
np.random.seed(350)
te_idx,tr_idx = helper.modified_split(enz_names)

In [35]:
X = np.array(list(map(list,X)))

In [36]:
total_length = len(X[0])

In [45]:
band_length = 100
motif_length = 7
band_step = 25
motif_step = 1

band_number = 0
curr_extreme = band_number*band_step + band_length

test_preds,test_acc,train_acc = dict(),dict(),dict()

while curr_extreme<total_length:
    X_sliced = X[:,[i for i in range(band_number*band_step,band_number*band_step+band_length)]]
    ctdModel = CTDModel(enz_names,X_sliced,y,tr_idx,te_idx,Feats='CT',k=motif_length,s=motif_step,random_seed=350)
    test_acc[band_number] = ctdModel.SVMobject.acc_test
    train_acc[band_number] = ctdModel.SVMobject.acc_train
    test_preds[band_number] = ctdModel.SVMobject.ypredtest
    

    band_number+=1
    curr_extreme = band_number*band_step + band_length    


In [46]:
train_acc

{0: 0.6206896551724138,
 1: 0.6896551724137931,
 2: 0.7471264367816092,
 3: 0.7586206896551724,
 4: 0.7586206896551724,
 5: 0.7586206896551724,
 6: 0.8735632183908046,
 7: 0.8735632183908046,
 8: 0.9425287356321839,
 9: 0.9310344827586207,
 10: 0.896551724137931,
 11: 0.8390804597701149,
 12: 0.7586206896551724,
 13: 0.7586206896551724,
 14: 0.7586206896551724,
 15: 0.7586206896551724,
 16: 0.7586206896551724,
 17: 0.7586206896551724}

In [47]:
test_acc

{0: 0.4074074074074074,
 1: 0.5925925925925926,
 2: 0.6296296296296297,
 3: 0.6296296296296297,
 4: 0.6296296296296297,
 5: 0.5185185185185185,
 6: 0.6666666666666666,
 7: 0.5925925925925926,
 8: 0.48148148148148145,
 9: 0.5185185185185185,
 10: 0.5925925925925926,
 11: 0.48148148148148145,
 12: 0.5185185185185185,
 13: 0.5185185185185185,
 14: 0.5185185185185185,
 15: 0.5185185185185185,
 16: 0.5185185185185185,
 17: 0.5185185185185185}

In [43]:
y[te_idx]

array([2, 2, 3, 2, 3, 2, 3, 2, 2, 2, 1, 3, 3, 3, 1, 1, 3, 3, 3, 3, 1, 3,
       1, 3, 1, 3, 2])

In [44]:
test_preds

{0: array([3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 1, 3, 3, 1, 3, 3, 1, 1, 3, 3, 1, 1,
        3, 3, 1, 1, 1]),
 1: array([3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 1, 1, 1, 3, 1, 3, 3, 1, 3,
        3, 3, 1, 3, 1]),
 2: array([3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 2, 1, 1, 3, 1, 3, 3, 1, 3,
        1, 3, 1, 3, 1]),
 3: array([3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 2, 1, 1, 3, 1, 3, 3, 1, 3,
        1, 3, 1, 3, 1]),
 4: array([3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 2, 1, 1, 3, 1, 3, 3, 1, 3,
        1, 3, 1, 3, 1]),
 5: array([3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 2, 3, 3, 3, 1, 3, 3, 3, 3,
        1, 3, 1, 3, 1]),
 6: array([1, 1, 3, 1, 3, 3, 3, 2, 1, 1, 1, 3, 3, 2, 1, 1, 3, 2, 3, 3, 1, 3,
        1, 3, 1, 3, 1]),
 7: array([1, 1, 3, 1, 3, 3, 3, 2, 1, 1, 1, 3, 3, 2, 1, 1, 2, 2, 3, 3, 1, 3,
        1, 3, 1, 2, 1]),
 8: array([1, 1, 2, 1, 3, 3, 3, 3, 1, 1, 1, 3, 2, 2, 1, 1, 2, 1, 3, 3, 1, 3,
        1, 3, 1, 2, 1]),
 9: array([1, 3, 2, 1, 3, 3, 3, 2, 1, 3, 1, 3, 2, 2, 1, 1, 1, 1, 3, 3, 1, 3,
     

In [9]:
from IndModels.models import Model

In [11]:
class CTDModel(Model):
    def __init__(self,enzs,X,y,tr_idx,te_idx,k=7,s=1,Feats='CTD',
                 SVM=True,GBC=False,NN=False,pca_components=40,
                 regCparam=20,kernparam='rbf',nestparam=15,lrateparam=0.5,
                 mdepthparam=3,ssampleparam=1,hlayer=(5,),lrateinit=0.1,
                 regparam=0.01,inc_count=True,probability=False,
                 classweight=None,optimizeQ=False,verboseQ=False,
                 random_seed=None):

        self.ctd = CTD(F=Feats)
        self.X_enc = self.get_ctd_feat(X,k,s)
        self.Xtrain,self.Xtest = self.X_enc[tr_idx,:],self.X_enc[te_idx,:]

        super().__init__(enzs,X,y,tr_idx,te_idx,SVM,GBC,NN,pca_components,regCparam,
             kernparam,nestparam,lrateparam,mdepthparam,ssampleparam,hlayer,
             lrateinit,regparam,random_seed,probability,classweight,optimizeQ,verboseQ)

        if SVM:
            self.SVMobject = self.get_SVM()
            self.model = self.SVMobject.model

        elif GBC:
            self.GBCobject=self.get_GBC()
            self.model=self.GBCobject.model

        elif NN:
            self.NNobject=self.get_NN()
            self.model=self.NNobject.model


        else:
            raise ValueError('No model initiated')
        
        pass

    def get_ctd_feat(self,X,motif_length=7,motif_step=1):
        len_seq = len(X[0])
        X_encoded = []
        for seq in X:
            seq_encoded = []
            curr_motif = 0
            curr_extreme = curr_motif*motif_step + motif_length

            while curr_extreme<len_seq:
                motif = seq[curr_motif*motif_step:curr_extreme]
                motif_feat = ctdEnc.get_ind_attr(motif)
                seq_encoded.extend(motif_feat)
                curr_motif+=1
                curr_extreme = curr_motif*motif_step + motif_length

            X_encoded.append(seq_encoded)

        return np.array(X_encoded)            

In [20]:
ctdmodel = CTDModel(enz_names,X[:,[i for i in range(400,500)]],y,tr_idx,te_idx)

In [21]:
ctdmodel.SVMobject.acc_test

0.5185185185185185

In [11]:
def get_ctdEnc(X_sliced,motif_length=7,motif_step=1):
    len_seq = len(X_sliced[0])
    X_encoded = []
    for seq in X_sliced:
        seq_encoded = []
        curr_motif = 0
        curr_extreme = curr_motif*motif_step + motif_length
        
        while curr_extreme<len_seq:
            motif = seq[curr_motif*motif_step:curr_extreme]
            motif_feat = ctdEnc.get_ind_attr(motif)
            seq_encoded.extend(motif_feat)
            curr_motif+=1
            curr_extreme = curr_motif*motif_step + motif_length
            
        X_encoded.append(seq_encoded)
        
    return np.array(X_encoded)

In [12]:
X_enc = get_ctdEnc(X[:,[i for i in range(0,100)]])

In [13]:
X_enc.shape

(116, 13671)

In [16]:
X_train = X_enc[tr_idx,:]
X_test = X_enc[te_idx,:]
y_train,y_test = y[tr_idx],y[te_idx]

In [18]:
svm = SVM(X_train,X_test,y_train,y_test)

-----Initial Model Evaluation-----
-----Training Accuracy:0.5862068965517241-----
-----Testing Accuracy:0.48148148148148145-----
